In [19]:
import psycopg2
import pandas as pd

# Credenciales de la base de datos
dbname = "DBInmerso"
user = "postgres"
password = "postgres"
host = "localhost"  # Cambia esto según tu configuración
port = "5432"

# Crear la cadena de conexión
connection_string = f"dbname={dbname} user={user} password={password} host={host} port={port}"

# Intentar establecer la conexión
try:
    connection = psycopg2.connect(connection_string)

    # Consulta para concatenar las tablas
    query_concatenar = """
        SELECT
            u.dni,
            u.nombre,
            u.apellido,
            u.edad,
            u.fecha_de_nacimiento,
            u.id_solicitud,
            u.usuario_solicitante,
            u.oficio_especial,
            r.ingresos,
            d.grado_dis,
            fn.tipo_fam_num,
            p.valoracion_patrimonio,
            p.coche,
            p.arrendador
        FROM
            usuarios u
        JOIN renta r ON u.dni = r.dni
        JOIN discapacidad d ON u.dni = d.dni
        JOIN familia_numerosa fn ON u.dni = fn.dni
        JOIN patrimonio p ON u.dni = p.dni;
    """

    # Crear un DataFrame con los resultados de la consulta concatenada
    df = pd.read_sql_query(query_concatenar, connection)

except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")

finally:
    # Cerrar la conexión en el bloque 'finally' para asegurar que se cierre
    if connection:
        connection.close()


#Ponderaciones
edad_weight = 0.1
discapacidad_weight = 0.1
familia_weight = 0.1
renta_wheigt = 0.1

df

C:\Users\DELL\AppData\Local\Temp\ipykernel_3204\3136081874.py:44: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query_concatenar, connection)


,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador
0,95822412,Juan,Borja,62,1936-07-20,52,True,False,2336.0,2,1,2000000.0,True,True
1,46913810,Adelia,Rey,70,1943-06-25,458,True,True,4247.0,2,2,441843.0,False,False
2,28728463,Angélica,Pol,61,1951-08-03,1386,True,True,3132.0,0,0,138429.0,False,True
3,83197857,Amor,Vazquez,60,1928-03-07,1210,False,False,3452.0,1,0,347455.0,False,True
4,66629388,Eliseo,Bilbao,57,1957-02-04,62,False,False,1517.0,0,2,409487.0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,69148126,Jordana,Lerma,57,1935-03-05,130,True,True,3718.0,2,0,47721.0,True,True
1496,45113084,Atilio,Machado,75,1945-11-29,26,False,False,2269.0,1,1,318972.0,False,True
1497,80439867,Salomé,Montaña,69,1928-02-29,1372,True,False,4993.0,0,2,293549.0,True,False
1498,55405943,Norberto,Trillo,88,1926-03-22,1072,False,False,1551.0,0,2,119297.0,True,False


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dni                    1500 non-null   object 
 1   nombre                 1500 non-null   object 
 2   apellido               1500 non-null   object 
 3   edad                   1500 non-null   int64  
 4   fecha_de_nacimiento    1500 non-null   object 
 5   id_solicitud           1500 non-null   int64  
 6   usuario_solicitante    1500 non-null   bool   
 7   oficio_especial        1500 non-null   bool   
 8   ingresos               1500 non-null   float64
 9   grado_dis              1500 non-null   int64  
 10  tipo_fam_num           1500 non-null   int64  
 11  valoracion_patrimonio  1500 non-null   float64
 12  coche                  1500 non-null   bool   
 13  arrendador             1500 non-null   bool   
dtypes: bool(4), float64(2), int64(4), object(4)
memory usage

### EDAD.SCORE

In [11]:
def puntaje_edad_nueva(edad):
    if edad < 60:
        return 1
    elif 60 <= edad < 70:
        return 20
    elif 70 <= edad < 80:
        return 30
    else:  # Edad >= 80
        return 40

def procesar_edad(df):
    # Copiar el DataFrame para evitar modificar el original
    df_edad = df.copy()

    # Aplicar la función a la columna 'edad' y crear la nueva columna 'score_edad'
    df_edad['score_edad'] = df_edad['edad'].apply(puntaje_edad_nueva)

    # Calcular el Min-Max Scaling para la columna 'score_edad'
    min_valor = df_edad['score_edad'].min()
    max_valor = df_edad['score_edad'].max()
    rango_deseado = 100
    min_deseado = 0

    df_edad['score_edad_scaled'] = ((df_edad['score_edad'] - min_valor) / (max_valor - min_valor)) * rango_deseado + min_deseado

    # Multiplicar 'score_edad_scaled' por el ponderador
    df_edad['score_edad_weighted'] = df_edad['score_edad_scaled'] * edad_weight
    
    # Devolver el DataFrame con las columnas 'dni', 'edad', 'score_edad', 'score_edad_scaled' ordenado por 'dni'
    return df_edad[['dni', 'edad', 'score_edad', 'score_edad_scaled','score_edad_weighted']].sort_values(by='dni')

# Ejemplo de uso:
# df_original es tu DataFrame original
df_edad = procesar_edad(df)
df_edad


,dni,edad,score_edad,score_edad_scaled,score_edad_weighted
55,10076758,75,30,74.358974,7.435897
1290,10171954,58,1,0.000000,0.000000
1192,10196231,79,30,74.358974,7.435897
608,10209961,77,30,74.358974,7.435897
348,10344303,73,30,74.358974,7.435897
...,...,...,...,...,...
1472,99794603,86,40,100.000000,10.000000
144,99913412,70,30,74.358974,7.435897
1286,99929175,76,30,74.358974,7.435897
24,99949389,72,30,74.358974,7.435897


### DISCAPACIDAD.SCORE

In [14]:
def puntaje_discapacidad(grado_dis):
    if grado_dis == 0:
        return 0
    elif grado_dis == 1:
        return 10
    else:  # grado_dis == 2
        return 20

def procesar_discapacidad(df):
    # Copiar el DataFrame para evitar modificar el original
    df_discapacidad = df.copy()

    # Aplicar la función a la columna 'edad' y crear la nueva columna 'score_edad'
    df_discapacidad['score_discapacidad'] = df_discapacidad['grado_dis'].apply(puntaje_discapacidad)

    # Calcular el Min-Max Scaling para la columna 'score_edad'
    min_valor = df_discapacidad['score_discapacidad'].min()
    max_valor = df_discapacidad['score_discapacidad'].max()
    rango_deseado = 100
    min_deseado = 0

    df_discapacidad['score_discapacidad_scaled'] = ((df_discapacidad['score_discapacidad'] - min_valor) / (max_valor - min_valor)) * rango_deseado + min_deseado

    df_discapacidad['score_discapacidad_weighted'] = df_discapacidad['score_discapacidad_scaled'] * discapacidad_weight
    
    # Devolver el DataFrame con las columnas 'dni', 'score_discapacidad', 'score_edad_scaled' ordenado por 'dni'
    return df_discapacidad[['dni', 'score_discapacidad', 'score_discapacidad_scaled','score_discapacidad_weighted']].sort_values(by='dni')

# Ejemplo de uso:
# df_original es tu DataFrame original
df_discapacidad = procesar_discapacidad(df)
df_discapacidad

,dni,score_discapacidad,score_discapacidad_scaled,score_discapacidad_weighted
55,10076758,0,0.0,0.0
1290,10171954,0,0.0,0.0
1192,10196231,20,100.0,10.0
608,10209961,10,50.0,5.0
348,10344303,0,0.0,0.0
...,...,...,...,...
1472,99794603,20,100.0,10.0
144,99913412,0,0.0,0.0
1286,99929175,0,0.0,0.0
24,99949389,10,50.0,5.0


### SCORE FAMILIA NUMEROSA

In [18]:
def puntaje_familia(tipo_fam_num):
    if tipo_fam_num == 0:
        return 0
    elif tipo_fam_num == 1:
        return 5
    else:  # tipo_fam_num == 2
        return 10

def procesar_familia(df):
    # Copiar el DataFrame para evitar modificar el original
    df_familia = df.copy()

    # Aplicar la función a la columna 'edad' y crear la nueva columna 'score_edad'
    df_familia['score_familia'] = df_familia['tipo_fam_num'].apply(puntaje_familia)

    # Calcular el Min-Max Scaling para la columna 'score_edad'
    min_valor = df_discapacidad['score_familia'].min()
    max_valor = df_discapacidad['score_familia'].max()
    rango_deseado = 100
    min_deseado = 0

    df_familia['score_familia_scaled'] = ((df_familia['score_familia'] - min_valor) / (max_valor - min_valor)) * rango_deseado + min_deseado

    df_familia['score_familia_weighted'] = df_familia['score_familia_scaled'] * familia_weight
    
    # Devolver el DataFrame con las columnas 'dni', 'score_discapacidad', 'score_edad_scaled' ordenado por 'dni'
    return df_discapacidad[['dni', 'score_familia', 'score_familia_scaled','score_familia_weighted']].sort_values(by='dni')

# Ejemplo de uso:
# df_original es tu DataFrame original
df_familia = procesar_discapacidad(df)
df_familia

,dni,score_discapacidad,score_discapacidad_scaled,score_discapacidad_weighted
55,10076758,0,0.0,0.0
1290,10171954,0,0.0,0.0
1192,10196231,20,100.0,10.0
608,10209961,10,50.0,5.0
348,10344303,0,0.0,0.0
...,...,...,...,...
1472,99794603,20,100.0,10.0
144,99913412,0,0.0,0.0
1286,99929175,0,0.0,0.0
24,99949389,10,50.0,5.0


### RENTA

In [20]:
df.sort_values(by= 'id_solicitud')

,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador
1264,80658723,Tania,Salvà,85,1923-12-31,2,False,False,2949.0,0,2,284909.0,True,False
327,64247457,Zaira,Quintanilla,87,1924-06-17,3,False,True,4670.0,0,1,124897.0,True,True
1110,22240547,Teófilo,Canet,61,1917-09-21,3,True,True,2291.0,2,1,250219.0,False,True
160,15957459,Dorotea,Pedrosa,82,1912-07-01,4,False,True,2717.0,1,1,461843.0,False,False
92,32097220,Valentina,Múñiz,79,1944-10-27,5,False,True,4046.0,1,0,463530.0,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,65177213,Maite,Hernando,84,1935-02-24,1494,True,False,1836.0,2,1,421358.0,True,True
1017,21523236,Febe,Rovira,62,1911-09-16,1496,True,True,4532.0,0,0,129803.0,False,True
247,72394301,Cristian,Pedrero,82,1934-09-22,1498,False,False,3134.0,1,0,210240.0,False,True
800,54383668,Benigna,Rincón,65,1953-11-17,1498,False,False,4416.0,1,0,399634.0,False,True


In [22]:
df[df.duplicated(subset=['id_solicitud'], keep=False)].sort_values(by='id_solicitud')

,dni,nombre,apellido,edad,fecha_de_nacimiento,id_solicitud,usuario_solicitante,oficio_especial,ingresos,grado_dis,tipo_fam_num,valoracion_patrimonio,coche,arrendador
1110,22240547,Teófilo,Canet,61,1917-09-21,3,True,True,2291.0,2,1,250219.0,False,True
327,64247457,Zaira,Quintanilla,87,1924-06-17,3,False,True,4670.0,0,1,124897.0,True,True
911,19976458,Severino,Sales,58,1932-10-29,14,True,True,2491.0,1,0,393866.0,False,True
917,75478707,Jordana,Tamayo,56,1939-07-18,14,False,False,3988.0,2,2,323385.0,True,True
119,97775215,Gil,Arenas,88,1934-03-10,17,True,True,1431.0,0,2,494824.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
636,45798645,Ariadna,Borrell,60,1910-02-15,1490,False,False,2812.0,0,1,331534.0,False,True
1440,49034245,Matías,Cabrero,84,1923-10-29,1491,False,False,4108.0,2,0,89630.0,True,False
849,80980044,Juan Antonio,Uribe,63,1942-03-20,1491,False,False,379.0,1,0,227092.0,True,False
800,54383668,Benigna,Rincón,65,1953-11-17,1498,False,False,4416.0,1,0,399634.0,False,True


In [23]:
#cogemos solo los valores unicos de la tabla usuarios
valores_unicos = df['id_solicitud'].unique().tolist()
df_solicitud = pd.DataFrame(valores_unicos, columns=['id_solicitud'])


# esta tabla tendrá el mismo tamañano que la de solicitudes

num_solicitud=len(df_solicitud)
print(num_solicitud)
data_solicitudes = []

# Generar 1000 registros
for _ in range(num_solicitud):
   anyo_solicitud = 2024
   usuarios_sol = 0
   renta_sol = 0

   data_solicitudes.append([anyo_solicitud,usuarios_sol,renta_sol])

solicitudes = pd.DataFrame(data_solicitudes, columns=['anyo_solicitud','usuarios_sol','renta_sol'])

# Concatenar los DataFrames con los ID solicitud de la tabla usuarios
df_solicitudes = pd.concat([df_solicitud, solicitudes], axis=1)
df_solicitudes

#cogemos la tabla de disponibilidad para que coincida 

col= ['id_hotel','fecha_disponibilidad_hab']

df_disponibilidad_col =df_disponibilidad[col].sort_values(by='id_hotel')
df_solicitudes1 = pd.concat([df_solicitudes, df_disponibilidad_col], axis=1)
df_solicitudes1.rename(columns={'id_hotel': 'primera_opcion','fecha_disponibilidad_hab': 'fecha_1op'}, inplace=True)


df_desordenado = df_solicitudes1.sample(frac=1).reset_index(drop=True)
df_solicitudes2 = pd.concat([df_desordenado, df_disponibilidad_col], axis=1)
df_solicitudes2.rename(columns={'id_hotel': 'segunda_opcion','fecha_disponibilidad_hab': 'fecha_2op'}, inplace=True)

df_desordenado2 = df_solicitudes2.sample(frac=1).reset_index(drop=True)
df_solicitudes_final = pd.concat([df_desordenado2, df_disponibilidad_col], axis=1)
df_solicitudes_final.rename(columns={'id_hotel': 'tercera_opcion','fecha_disponibilidad_hab': 'fecha_3op'}, inplace=True)


df_solicitudes_final= df_solicitudes_final.head(num_solicitud)
df_solicitudes_final

943


NameError: name 'df_disponibilidad' is not defined